In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertModel
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_data = pd.read_csv('/content/twitter_training.csv')
valid_data = pd.read_csv('/content/twitter_validation.csv')

In [ ]:
x_train = train_data.iloc[:, -1]
y_train = train_data.iloc[:, 2]

x_valid = valid_data.iloc[:, -1]
y_valid = valid_data.iloc[:, 2]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
x_train = x_train.astype(str)
x_valid = x_valid.astype(str)

x_train_encoded = tokenizer(x_train.tolist(), padding=True, truncation=True, return_tensors = 'tf')
x_valid_encoded = tokenizer(x_valid.tolist(), padding=True, truncation=True, return_tensors = 'tf')

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = LabelEncoder().fit_transform(y_train)
y_valid_encoded = LabelEncoder().fit_transform(y_valid)

In [ ]:
#Model Definition: BERT-based Sentiment Analysis Model
class SentimentModel(tf.keras.Model):
  def __init__(self):
    super(SentimentModel, self).__init__()
    self.bert = TFBertModel.from_pretrained('bert-base-uncased')
    self.dense = tf.keras.layers.Dense(1, activation = 'sigmoid')

  def call(self, inputs):
    outputs = self.bert(inputs)
    return self.dense(outputs[1])

In [19]:
# Initialize model
model = SentimentModel()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# train the model
history = model.fit(
    x_train_encoded['input_ids'], y_train_encoded, validation_data = (x_valid_encoded['input_ids'], y_valid_encoded), epochs = 3, batch_size = 32)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
  20/2334 ━━━━━━━━━━━━━━━━━━━━ 24:30:37 38s/step - accuracy: 0.2897 - loss: -0.8672

KeyboardInterrupt: 

In [ ]:
# Model Evaluation
# Evaluation
accuracy = model.evaluate(X_valid_encoded['input_ids'], y_valid_encoded)
print(f'Validation Accuracy: {accuracy[1] * 100:.2f}%')

In [ ]:
# Save the model
model.save('sentiment_model.h5')